In [1]:
# Задание на повторение материала предыдущего семестра

In [2]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [3]:
# Генерируем уникальный seed
my_code = "Паращенко"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("../datasets/russian_demography.csv")

In [5]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [6]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [7]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [8]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [9]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [10]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [11]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [12]:
sc_train

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,1.0,0.576227,0.422535,0.252336,0.621013,0.636124,Nenets Autonomous Okrug
1,0.222222,0.258398,0.122066,0.528037,0.779766,0.529862,Belgorod Oblast
2,0.777778,0.245478,0.187793,0.616822,0.545535,0.616348,Ryazan Oblast
3,0.148148,0.312661,0.220657,0.528037,0.826719,0.386421,Republic of Adygea
4,0.222222,0.263566,0.150235,0.546729,0.639794,0.692419,Perm Krai
...,...,...,...,...,...,...,...
1384,0.111111,0.209302,0.13615,0.630841,0.868001,0.459591,Kursk Oblast
1385,0.555556,0.237726,0.164319,0.602804,0.358965,0.73764,Samara Oblast
1386,0.0,0.431525,0.276995,0.369159,0.660524,0.836256,Sverdlovsk Oblast
1387,0.555556,0.242894,0.211268,0.649533,0.304926,0.676994,Arkhangelsk Oblast


In [13]:
# Вспоминаем алгоритмы решения задачи регрессии: линейную регрессию и метод k ближайших соседей
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

In [14]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [15]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [16]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.020546437602832708
0.030636937983122295
0.02046660919272234
0.030636937983122295
0.011524284406795036
0.013032362927390736
0.013762315740806868


In [17]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [18]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.010884500074527207


In [19]:
# Вспоминаем алгоритмы решения задачи классификации:
# логистическую регрессию, наивный байесовский классификатор и (снова) метод k ближайших соседей
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga'))
c_models.append(LogisticRegression(penalty='l1', solver='saga'))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

In [20]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [21]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:509: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [22]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.24448031396008316
0.12005023772954132
0.05664212115870458
0.08317595381365286
0.05664212115870458
0.01888900710704178
0.01887401450280291
0.018873591716940717
0.386284353920714
0.2677704650095218
0.2165122057877944


In [23]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model

MultinomialNB()

In [24]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.02077957539374307


In [25]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [26]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,1.0,0.576227,0.422535,0.252336,0.621013,0.636124
1,0.222222,0.258398,0.122066,0.528037,0.779766,0.529862
2,0.777778,0.245478,0.187793,0.616822,0.545535,0.616348
3,0.148148,0.312661,0.220657,0.528037,0.826719,0.386421
4,0.222222,0.263566,0.150235,0.546729,0.639794,0.692419
...,...,...,...,...,...,...
458,0.148148,0.255814,0.13615,0.546729,0.678242,0.753593
459,0.111111,0.330749,0.206573,0.481308,0.688696,0.602241
460,0.703704,0.235142,0.197183,0.621495,0.464741,0.499011
461,0.296296,0.268734,0.084507,0.471963,0.5,0.711536


In [27]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [28]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2925907547250885
0.2612302790798771
0.23451067012188082
0.2385183380769227


In [29]:
# Выбираем лучшую модель
i_min = sils.index(min(sils))
best_k_model = k_models[i_min]
print(best_k_model)
print(sils[i_min])



KMeans(n_clusters=20)
0.23451067012188082


In [57]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25)
]

In [58]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[ElasticNet(), KNeighborsRegressor(n_neighbors=10), ElasticNet(l1_ratio=0.75), KNeighborsRegressor(n_neighbors=20)]


In [59]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("../datasets/weather.csv")
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,"468,0","0,0","24,9","31,0","100,0","4,0","0,0","993,8","4,0","-3,0","-0,3","-0,4"
1,"140,0","0,0","27,9","22,0","20,0","1,0","0,0","1004,1","4,0","-2,0","0,6","-1,0"
2,"464,0","0,0","25,6","28,0","20,0","2,0","1,0","1001,2","4,0","-7,0","-1,8","-0,3"
3,"467,0","0,0","26,7","26,0","100,0","3,0","0,0","992,3","4,0","-1,0","1,2","-1,3"
4,"138,0","0,0","29,1","22,0","20,0","1,0","0,0","1001,3","4,0","-2,0","2,5","-1,3"
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,"474,0","0,1","6,4","95,0","4,0","1,0","3,0","999,9","4,0","-1,0","5,9","1,5"
1435,"65,0","0,0","6,4","93,0","10,0","4,0","3,0","983,9","4,0","-1,0","5,2","3,1"
1436,"159,0","0,1","8,5","83,0","10,0","2,0","1,0","969,7","4,0","1,0","1,4","1,9"
1437,"99,0","0,1","7,5","91,0","10,0","3,0","3,0","995,4","4,0","-4,0","2,7","1,7"


In [60]:
data['water_level'] = data['water_level'].str.replace(',','.')
data['precipitation'] = data['precipitation'].str.replace(',','.')
data['temperature'] = data['temperature'].str.replace(',','.')
data['humidity'] = data['humidity'].str.replace(',','.')
data['visibility'] = data['visibility'].str.replace(',','.')
data['wind'] = data['wind'].str.replace(',','.')
data['weather'] = data['weather'].str.replace(',','.')
data['pressure'] = data['pressure'].str.replace(',','.')
data['fire'] = data['fire'].str.replace(',','.')                       
data['wl_change'] = data['wl_change'].str.replace(',','.')                       
data['temp_change'] = data['temp_change'].str.replace(',','.')                      
data['pressure_change'] = data['pressure_change'].str.replace(',','.')                         

In [61]:
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [62]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = ['water_level']
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['precipitation', 'wl_change', 'humidity', 'temp_change', 'wind']


In [63]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.


In [64]:
lb =  x_labels + y_label
data=data[list(lb)]
data

,precipitation,wl_change,humidity,temp_change,wind,water_level
0,0.0,-3.0,31.0,-0.3,4.0,468.0
1,0.0,-2.0,22.0,0.6,1.0,140.0
2,0.0,-7.0,28.0,-1.8,2.0,464.0
3,0.0,-1.0,26.0,1.2,3.0,467.0
4,0.0,-2.0,22.0,2.5,1.0,138.0
...,...,...,...,...,...,...
1434,0.1,-1.0,95.0,5.9,1.0,474.0
1435,0.0,-1.0,93.0,5.2,4.0,65.0
1436,0.1,1.0,83.0,1.4,2.0,159.0
1437,0.1,-4.0,91.0,2.7,3.0,99.0


In [65]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [66]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [67]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = lb

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['precipitation', 'wl_change', 'humidity',
                                  'temp_change', 'wind', 'water_level'])])

In [68]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [69]:
# Устанавливаем названия столбцов
column_names = lb
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [70]:
sc_train

,precipitation,wl_change,humidity,temp_change,wind,water_level
0,0.00000,0.383495,0.252747,0.486989,0.8,0.705314
1,0.00000,0.383495,0.780220,0.386617,0.1,0.339775
2,0.00000,0.364078,0.186813,0.572491,0.1,0.337359
3,0.43750,0.393204,0.802198,0.334572,0.0,0.231884
4,0.00625,0.378641,0.758242,0.475836,0.6,0.329308
...,...,...,...,...,...,...
858,0.00000,0.393204,0.956044,0.423792,0.0,0.242351
859,0.00000,0.393204,0.461538,0.535316,0.2,0.380837
860,0.00000,0.373786,0.428571,0.550186,0.0,0.194042
861,0.00000,0.325243,0.252747,0.609665,0.1,0.497585


In [71]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_label]
y_test = sc_test[y_label]
y_val = sc_val[y_label]

In [72]:
# Обучаем модели
for model in my_models1:
    model.fit(x_train, y_train)

In [73]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in my_models1:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.029627475890015766
0.028874615800201626
0.029627475890015766
0.027652951964679873


In [74]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = mses.index(min(mses))
best_my_model = my_models1[i_min]
best_my_model

KNeighborsRegressor(n_neighbors=20)

In [75]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.11023649720281717


In [76]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25)
]

In [77]:
# Выбор моделей для задания
n = 5
my_models = random.sample(c_models, n)
print(my_models)

[KNeighborsClassifier(), KNeighborsClassifier(n_neighbors=15), LogisticRegression(penalty='none', solver='saga'), KNeighborsClassifier(n_neighbors=20), MultinomialNB()]


In [78]:
# Загрузим данные для задачи классификации
data = pd.read_csv("../datasets/zoo2.csv")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [79]:
data.drop(['animal_name'], axis='columns', inplace=True)

In [80]:
data.head()

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3


In [81]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = ['class_type']
x_labels = random.sample(columns[:-1], n_x)
lb =  x_labels + y_label
print(x_labels)

['fins', 'milk', 'legs', 'aquatic', 'airborne', 'feathers', 'predator', 'eggs']


In [82]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [83]:
data=data[list(lb)]
data.head()

,fins,milk,legs,aquatic,airborne,feathers,predator,eggs,class_type
0,0,0,4,1,0,0,0,1,3
1,0,0,4,0,0,0,0,1,3
2,0,0,4,0,0,0,1,1,3
3,0,0,4,0,0,0,1,1,3
4,0,0,4,0,0,0,0,1,3


In [84]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(data))
print(val_test_size)

9


In [85]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

25 9 9


In [86]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = x_labels

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['fins', 'milk', 'legs', 'aquatic', 'airborne',
                                  'feathers', 'predator', 'eggs'])])

In [87]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [88]:
# Устанавливаем названия столбцов
column_names = lb
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [89]:
sc_train

,fins,milk,legs,aquatic,airborne,feathers,predator,eggs,class_type
0,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,4.0
1,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,6.0
2,0.0,0.0,0.666667,1.0,0.0,0.0,1.0,0.0,3.0
3,0.0,0.0,0.666667,1.0,0.0,0.0,0.0,0.0,5.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,3.0
5,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,0.0,6.0
6,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,6.0
7,0.0,0.0,0.666667,1.0,0.0,0.0,0.0,0.0,5.0
8,0.0,0.0,0.666667,0.0,0.0,0.0,1.0,0.0,3.0
9,0.0,0.0,1.000000,0.0,1.0,0.0,0.0,0.0,6.0


In [90]:
x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_label])
y_test = np.ravel(sc_test[y_label])
y_val = np.ravel(sc_val[y_label])

In [92]:
# Обучаем модели
for model in my_models:
    model.fit(x_train, y_train)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [93]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in my_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.4351851851851851
0.2222222222222222
0.8962962962962963
0.02222222222222222
0.2857142857142857


In [94]:
# Выбираем лучшую модель # Укажите, какая модель решает задачу лучше других.
i_min = f1s.index(min(f1s))
best_my_models = my_models[i_min]
best_my_models

KNeighborsClassifier(n_neighbors=20)

In [96]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_my_models.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.16666666666666666
